# Install Dependencies

In [130]:
!pip install zarr
!pip install pandas
!pip install seaborn
!pip install tqdm
!pip install iso8601

  Obtaining dependency information for iso8601 from https://files.pythonhosted.org/packages/12/e4/032a8569b75a79057f0c72b4f4a69434fd21922bce7bfe054e0e9f367861/iso8601-2.0.0-py3-none-any.whl.metadata


In [26]:
import s3fs
import zarr
import json
from tqdm import tqdm
import pandas as pd

import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
from pprint import pprint

%matplotlib inline     
sns.set(color_codes=True)

# Grab Wifire Simulations Run Data 

In [27]:
endpoint = 'https://wifire-data.sdsc.edu:9000'
access_key = 'bp3d-perf'
secret_key = 'blvWRaejB1aTXPx6CqZ2GVxYbmWOSCDL'

fs = s3fs.S3FileSystem(key=access_key,
    secret=secret_key,
    client_kwargs={
        'endpoint_url': endpoint,
        'verify': False
    },
    skip_instance_cache=False
)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

name = 'quicfire.zarr'
bucket = 'burnpro3d/d'
# path = '00/16/run_001676e4-ed7a-42e4-abc5-9c6f1dc122df'
# with fs.open( bucket + '/' + path + '/' + name + '/.zattrs') as f:
#         data=json.load(f)

root = list(fs.ls(bucket))
simulation_paths = []

In [28]:
def get_paths(bucket):
    global fs
    paths = fs.ls(bucket)
    for path in tqdm(paths[1:2]):
        paths = fs.ls(path)
        for path in paths:
            get_child_directories(path)

def get_child_directories(path):
    global fs
    global simulation_paths
    paths = fs.ls(path)
    for p in paths:
        if "run_" in p:
            simulation_paths.append(p)

In [65]:
get_paths(bucket)
len(simulation_paths)

100%|████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 48.00it/s]


90

# Data Exploration

In [207]:
KEEP_ATTRIBUTES = {
    'path': lambda d: None,
    'canopy_moisture': lambda d: d['canopy_moisture'],
    # 'dz',
    'extent': lambda d: d['extent'],
    'extent_fmt': lambda d: d['extent_fmt'],
    'fire_grid': lambda d: d['fire_grid'],
    'fuel': lambda d: d['fuel'],
    'ignition': lambda d: d['ignition'],
    'output': lambda d: d['output'],
    'resolution': lambda d: d['resolution'],
    'resolution_units': lambda d: d['resolution_units'],
    'run_binary': lambda d: d['run_binary'],
    'run_end': lambda d: d['run_end'],
    'run_max_mem_rss_bytes': lambda d: d['run_max_mem_rss_bytes'],
    'run_start': lambda d: d['run_start'],
    'seed': lambda d: d['seed'],
    'sim_time': lambda d: d['sim_time'],
    'surface_moisture': lambda d: d['surface_moisture'],
    'threads': lambda d: d['threads'],
    'timestep': lambda d: d['timestep'],
    'topo': lambda d: d['topo'],
    'wind_direction': lambda d: d['wind_direction'],
    'wind_speed': lambda d: d['wind_speed']
}
filenotfound = []

def get_df(count):
    global simulation_paths, KEEP_ATTRIBUTES, incomplete, filenotfound

    df = pd.DataFrame([], columns=KEEP_ATTRIBUTES.keys() )
    i = 0
    for p in tqdm(simulation_paths[:count]):
        try:
            with fs.open(p + '/' + name + '/.zattrs') as f:
                data=json.load(f)
            # print("list:",[KEEP_ATTRIBUTES[i](data) for i in KEEP_ATTRIBUTES.keys()])
            
            row = []
            for k,expr in KEEP_ATTRIBUTES.items():
                try:
                    value = KEEP_ATTRIBUTES[k](data)
                except KeyError:
                    value = None
                row.append(value)
            row[0] = p
            df.loc[i] = row
            i+=1
        except FileNotFoundError as e:
            filenotfound.append(p)
            print("FileNotFound error on path {", e,"}")
    return df

pd.set_option('display.max_columns', None)

df = get_df(20)

 25%|██████████████████▊                                                        | 5/20 [00:00<00:00, 20.95it/s]

FileNotFound error on path { burnpro3d/d/01/05/run_01053f2b-153c-44fc-bdfa-77912390e226/quicfire.zarr/.zattrs }


100%|██████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 12.35it/s]


,path,canopy_moisture,extent,extent_fmt,fire_grid,fuel,ignition,output,resolution,resolution_units,run_binary,run_end,run_max_mem_rss_bytes,run_start,seed,sim_time,surface_moisture,threads,timestep,topo,wind_direction,wind_speed
0,burnpro3d/d/01/02/run_010231d9-ccf4-4fc3-b50f-...,1.0,"[1120178.154045004, 913143.8979546748, 1120778...","[[x1, y1], [x2, y2]]",None,"{'density': None, 'dx': 2, 'dy': 2, 'height': ...","{'dat': [' igntype= 4 ', ' &aerial...","{'emissions': False, 'energy_atmos': False, 'f...","[1, 2, 2]",meters,/quicfire/quicfire_Jan2022,2022-07-13T00:57:37.108269-07:00,523172,2022-07-13T00:53:38.698521-07:00,-1,1000,0.211781,1,10,None,253.810814,9.039290
1,burnpro3d/d/01/02/run_0102f75c-8d04-46b6-86ea-...,1.0,"[1120178.154045004, 913143.8979546748, 1120778...","[[x1, y1], [x2, y2]]",None,"{'density': None, 'dx': 2, 'dy': 2, 'height': ...","{'dat': [' igntype= 5 ', ' &atv...","{'emissions': False, 'energy_atmos': False, 'f...","[1, 2, 2]",meters,/quicfire/quicfire_Jan2022,2022-07-13T00:27:08.501441-07:00,523068,2022-07-13T00:14:41.343688-07:00,-1,1000,0.000000,1,10,None,264.614715,4.550601
2,burnpro3d/d/01/06/run_01062a0e-e7ee-4420-aea2-...,0.1,"[0, 500, 500, 0]","[[x1, y1], [x2, y2]]",None,"{'density': 0.7, 'dx': 2, 'dy': 2, 'height': 0...",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2, 2]",meters,/quicfire/quicfire-v5.0.0,2022-11-16T04:54:10.094242-08:00,301760,2022-11-16T04:53:47.858409-08:00,-1,5,0.050000,1,100,None,270.000000,5.000000
3,burnpro3d/d/01/07/run_010761cc-2e4f-4206-9eb8-...,1.0,"[-1019.635396216905, 587.5323697931051, 780.36...","[[x1, y1], [x2, y2]]",None,"{'density': None, 'dx': 2, 'dy': 2, 'height': ...","{'dat': [' igntype= 4 ', ' &aerial...","{'emissions': False, 'energy_atmos': False, 'f...","[2, 2, 1]",meters,/quicfire/quicfire_Jan2022,None,None,2022-04-04T18:37:50.289153-07:00,-1,3200,0.050000,8,None,None,225.000000,2.000000
4,burnpro3d/d/01/08/run_010827f3-e59d-422e-9f42-...,1.0,"[0, 600, 600, 0]","[[x1, y1], [x2, y2]]",None,"{'density': 0.7, 'dx': 2, 'dy': 2, 'height': 1...","{'dat': [' igntype= 5 ', ' &atv...","{'emissions': False, 'energy_atmos': True, 'fi...","[2, 2]",meters,/ceph/QF_Versions/QF_1-6-22/source_code/compil...,2022-02-28T16:01:07.748676-08:00,654512,2022-02-28T15:49:57.081434-08:00,-1,600,0.100000,None,1,{'total_startup_iters': 0},280.000000,10.000000
5,burnpro3d/d/01/0e/run_010e74fc-2298-49a0-b5a1-...,1.0,"[0, 600, 600, 0]","[[x1, y1], [x2, y2]]",None,"{'density': 0.7, 'dx': 2, 'dy': 2, 'height': 1...","{'dat': None, 'id': 'fff84ed4-081a-4e4a-af98-4...","{'emissions': False, 'energy_atmos': False, 'f...","[2, 2]",meters,/quicfire/quicfire-v5.1.1,2023-01-07T17:38:59.939612-08:00,None,2023-01-07T17:37:48.212893-08:00,-1,600,0.050000,1,None,{'total_startup_iters': 0},270.000000,1.000000
6,burnpro3d/d/01/15/run_0115cd68-a700-4fcd-ac79-...,1.0,"[-1019.635396216905, 587.5323697931051, 780.36...","[[x1, y1], [x2, y2]]",None,"{'density': None, 'dx': 2, 'dy': 2, 'height': ...","{'dat': [' igntype= 4 ', ' &aerial...","{'emissions': False, 'energy_atmos': False, 'f...","[2, 2, 1]",meters,/quicfire/quicfire_Jan2022,2022-04-05T16:05:12.817134-07:00,None,2022-04-05T12:04:45.504844-07:00,-1,3200,0.350000,8,None,None,195.000000,2.000000
7,burnpro3d/d/01/15/run_0115dbde-51cc-4d14-8a0f-...,0.1,"[0, 500, 500, 0]","[[x1, y1], [x2, y2]]",None,"{'density': 0.7, 'dx': 2, 'dy': 2, 'height': 0...",None,"{'emissions': False, 'energy_atmos': False, 'f...","[2, 2]",meters,/quicfire/quicfire-v4.0.1,2022-11-07T16:19:18.304062-08:00,457656,2022-11-07T16:19:03.839372-08:00,-1,5,0.050000,8,100,None,270.000000,5.000000
8,burnpro3d/d/01/17/run_01176c66-ae94-4a5d-bc74-...,1.0,"[-2048049, 1866939, -2046749, 1866139]","[[x1, y1], [x2, y2]]",None,"{'density': None, 'dx': 2, 'dy': 2, 'height': ...","{'dat': [' igntype= 4 ', ' &aerial...","{'emissions': False, 'energy_atmos': False, 'f...","[1, 2, 2]",meters,/quicfire/quicfire_Jan2022,2022-03-31T15:13:40.406424-07:00,3566072,2022-03-31T13:30:39.98

In [169]:
from datetime import datetime
import iso8601

date_str = '2022-07-13T00:57:37.108269-07:00'
iso8601.parse_date(date_str)

datetime.datetime(2022, 7, 13, 0, 57, 37, 108269, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200), '-07:00'))

In [33]:
# these are the top level directories, all 200
p = 'burnpro3d/d/00/0d/run_000d836a-e4d0-46c2-8ce6-021cf9ee4e9c'
with fs.open(p + '/' + name + '/.zattrs') as f:
                data=json.load(f)
pprint(data,compact=True,depth=2)
# print(data["resolution"])
# pprint(list(data.keys()))

{'canopy_moisture': 1.5,
 'dz': None,
 'extent': [939794.0, 1235562.0, 941860.0, 1233570.0],
 'extent_fmt': '[[x1, y1], [x2, y2]]',
 'fire_grid': None,
 'fuel': {'density': None,
          'dx': 2,
          'dy': 2,
          'height': 0.5,
          'id': '6831c3cc-883c-4633-a5c0-843c1b4ca723',
          'xlen': None,
          'ylen': None},
 'ignition': {'dat': [...], 'perc': 100},
 'output': {'emissions': False,
            'energy_atmos': False,
            'fire_energy': False,
            'fuels_dens': True,
            'fuels_moist': False,
            'plume_locs': False,
            'plume_traj': False,
            'qu_avg_wind': False,
            'qu_wind': False,
            'reaction_rate': False,
            'steps_fire': 120,
            'steps_wind': 100,
            'thermal_rad': False,
            'vert_mass_burnt': False,
            'wind': True},
 'resolution': [1.0, 2.0, 2.0],
 'resolution_units': 'meters',
 'run_binary': '/quicfire/quicfire-v4.0.1',
 'run_end'

# Next Step

In [12]:
# pandas group by version in dataframe

# caerful we have ign oor fuel object
# only look for ones that say run

In [13]:
# scan top directory 
# ls bucket but max step is 01